In [1]:
# !/bin/python3
# https://adventofcode.com/2022/day/16

%load_ext lab_black

In [2]:
import re
import sys

from copy import copy
from itertools import combinations
from itertools import zip_longest
from utils import read_input


pattern = re.compile("Valve (.*) has flow rate=(\d+); tunnels? leads? to valves? (.*)")

In [3]:
def parse(line):
    match = pattern.search(line).groups(0)
    return match[0], int(match[1]), match[2].split(", ")


def make_map(input):
    valves = {}
    for name, rate, destinations in input:
        valves[name] = {"rate": rate, "destinations": destinations}

    for valve, data in valves.items():
        data["distances"] = dijkstra(valves, valve)

    return valves


def dijkstra(valves, start):
    distances = {}
    unvisited = list(valves.keys())

    for node in unvisited:
        distances[node] = sys.maxsize
    distances[start] = 0

    while unvisited:
        curr = None
        for node in unvisited:
            if curr == None or distances[node] < distances[curr]:
                curr = node

        for node in valves[curr]["destinations"]:
            value = distances[curr] + 1
            if value < distances[node]:
                distances[node] = value

        unvisited.remove(curr)

    return distances

In [4]:
def get_max_pressure(valves, useful, time, start="AA", pressure=0, max_pressure=0):
    for valve in useful:
        curr_time = time - valves[start]["distances"][valve] - 1
        if curr_time < 0:
            continue

        curr_pressure = pressure + (curr_time * valves[valve]["rate"])
        if curr_pressure > max_pressure:
            max_pressure = curr_pressure

        curr_useful = copy(useful)
        curr_useful.remove(valve)

        max_pressure = get_max_pressure(
            valves,
            curr_useful,
            curr_time,
            start=valve,
            pressure=curr_pressure,
            max_pressure=max_pressure,
        )
    return max_pressure

In [5]:
def part_a(valves):
    useful = [name for name, data in valves.items() if data["rate"] > 0]
    return get_max_pressure(valves, useful, 30)

In [6]:
def part_b(valves):
    useful = set(name for name, data in valves.items() if data["rate"] > 0)
    total = len(useful)
    tried = {}

    max_pressure = 0
    for me in range((total // 2) - 1, (total // 2) + 1):
        elephant = total - me
        for my_valves in combinations(useful, me):
            my_valves = tuple(my_valves)
            if my_valves in tried:
                continue
            tried[my_valves] = True

            elephant_valves = (valve for valve in useful if valve not in my_valves)
            if elephant_valves in tried:
                continue
            tried[elephant_valves] = True

            pressure = get_max_pressure(valves, list(my_valves), 26)
            pressure += get_max_pressure(valves, list(elephant_valves), 26)

            if pressure > max_pressure:
                max_pressure = pressure
    return max_pressure

In [7]:
valves = make_map(read_input(parent=__vsc_ipynb_file__, val_type=parse, sample="a"))

print("part a:", part_a(valves))
print("part b:", part_b(valves))

part a: 1651
part b: 1707


In [8]:
valves = make_map(read_input(parent=__vsc_ipynb_file__, val_type=parse))

print("part a:", part_a(valves))
print("part b:", part_b(valves))

part a: 1850
part b: 2306
